# Création et évaluation de solutions<a class="anchor" id="top"></a>

Dans ce bloc-notes, vous allez entraîner plusieurs modèles en utilisant en particulier Amazon Personalize : 

1. Personnalisation de l'utilisateur – quels articles sont les plus pertinents pour un utilisateur spécifique.
1. Articles similaires – pour un article donné, quels sont ceux qui lui sont similaires.
1. Classement personnalisé – pour un utilisateur et une collection d'articles, dans quel ordre sont-ils les plus pertinents ?

## Plan

1. [Introduction](#intro)
1. [Créer des solutions](#solutions)
1. [Évaluer les solutions](#eval)
1. [Utilisation des métriques d'évaluation](#use)
1. [Stockage des variables utiles](#vars)

## Introduction <a class="anchor" id="intro"></a>

En résumé, pour la plupart, les algorithmes d'Amazon Personalize (appelés recettes) cherchent à résoudre différentes tâches, expliquées ici :

1. **Personnalisation de l'utilisateur** – Nouvelle version qui prend en charge TOUS les flux HRNN/besoins de personnalisation de l'utilisateur ; ce sera ce que nous utilisons ici.
1. **HRNN et métadonnées HRNN** – Recommande des articles en fonction des interactions précédentes de l'utilisateur avec les articles.
1. **HRNN-Coldstart** – Recommande de nouveaux articles pour lesquels les données d'interaction ne sont pas encore disponibles.
1. **Classement personnalisé** – Prend une collection d'articles et les ordonne dans l'ordre probable d'intérêt en utilisant une approche de type HRNN.
1. **SIMS (Éléments similaires)** – À partir d'un article donné, recommande d'autres articles avec lesquels les utilisateurs ont également interagi.
1. **Compte de popularité** – Recommande les articles les plus populaires, si HRNN ou métadonnées HRNN n'ont pas de réponse – cela est renvoyé par défaut.

Quel que soit le cas d'utilisation, les algorithmes partagent tous une base d'apprentissage sur les données d'interaction entre l'utilisateur et l'article, qui sont définies par trois attributs fondamentaux :

1. **UserID** – L'utilisateur qui a interagi
1. **ItemID** – Article avec lequel l'utilisateur a interagi
1. **Horodatage** – L'heure à laquelle l'interaction s'est produite

Nous prenons également en charge les types d'événements et les valeurs d'événements définis par :

1. **Type d'événement​​** – Étiquette catégorique d'un événement (parcourir, acheté, évalué, etc.).
1. **Valeur de l'événement** – Une valeur correspondant au type d'événement qui s'est produit. En règle générale, nous recherchons des valeurs normalisées entre 0 et 1 relatives aux types d'événements. Par exemple, s'il y a trois phases pour terminer une transaction (cliqué, ajouté au panier et acheté), alors il y aura une valeur event_value pour chaque phase comme 0,33, 0,66 et 1,0 respectivement.

Les champs type d'événement et valeur d'événement sont des données supplémentaires qui peuvent être utilisées pour filtrer les données envoyées en vue de l'entraînement du modèle de personnalisation. Dans cet exercice particulier, nous n'aurons pas de type d'événement ou de valeur d'événement. 

Pour exécuter ce bloc-notes, vous devez avoir exécuté les bloc-notes précédents, `01_Validating_and_Importing_User_Item_Interaction_Data` et `02_Validating_and_Importing_Item_Metadata.ipynb`, dans lesquels vous avez créé un jeu de données et importé des données de métadonnées d'interaction et d'articles dans Amazon Personalize. À la fin de ce bloc-notes, vous avez sauvegardé certaines des valeurs des variables, que vous devez maintenant charger dans ce bloc-notes.

In [1]:
%store -r

## Créer des solutions <a class="anchor" id="solutions"></a>
[Retour au début](#top)

Dans ce bloc-notes, vous allez créer des solutions avec les recettes suivantes :

1. Personnalisation de l'utilisateur
1. SIMS
1. Classement personnalisé

La recette Compte de popularité est la solution la plus simple disponible dans Amazon Personalize et ne doit être utilisée que comme solution de secours ; elle ne sera donc pas non plus couverte dans ce bloc-notes.

Comme pour le précédent bloc-notes, commencez par importer les paquets pertinents et configurez une connexion à Amazon Personalize à l'aide du kit SDK.

In [2]:
import time
from time import sleep
import json

import boto3

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

Dans Amazon Personalize, une variation spécifique d'un algorithme est appelée une recette. Des recettes différentes conviennent à des situations différentes. Un modèle entraîné s'appelle une solution, et chaque solution peut avoir de nombreuses versions qui se rapportent à un volume de données spécifique au moment où le modèle a été entraîné.

Tout d'abord, nous allons énumérer toutes les recettes qui sont prises en charge. Cela vous permettra d'en sélectionner une et de l'utiliser pour créer votre modèle.

In [4]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 8, 25, 16, 57, 0, 148000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 8, 25, 16, 57, 0, 148000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 8, 25, 16, 57, 0, 148000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   's

Le résultat est juste une représentation JSON de tous les algorithmes mentionnés dans l'introduction.

Ensuite, nous sélectionnerons des recettes spécifiques et créerons des modèles avec elles.

### Personnalisation de l'utilisateur
La recette Personnalisation de l'utilisateur (aws-user-personalization) est optimisée pour tous les scénarios de recommandation USER_PERSONALIZATION. Lorsque vous recommandez des articles, elle utilise l'exploration automatique des articles.

Avec l'exploration automatique, Amazon Personalize teste automatiquement les différentes recommandations d'articles, découvre comment les utilisateurs interagissent avec ces articles recommandés et stimule les recommandations pour les articles qui favorisent un meilleur engagement et une meilleure conversion. Cela améliore la découverte des articles et l'engagement lorsque vous disposez d'un catalogue qui évolue rapidement, ou lorsque de nouveaux articles, tels que des articles d'actualité ou des promotions, sont plus pertinents pour les utilisateurs lorsqu'ils sont frais.

Vous pouvez équilibrer le niveau d'exploration (où les articles présentant moins de données d'interaction ou de pertinence sont recommandés plus fréquemment) et le niveau d'exploitation (où les recommandations sont basées sur ce que nous savons ou sur la pertinence). Amazon Personalize ajuste automatiquement les recommandations futures en fonction des retours implicites des utilisateurs.

Tout d'abord, sélectionnez la recette en trouvant l'ARN dans la liste des recettes ci-dessus.

In [5]:
user_personalization_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

#### Créer la solution

Tout d'abord, vous créez une solution à l'aide de la recette. Même si vous fournissez le jeu de données ARN dans cette étape, le modèle n'est pas encore entraîné. Considérez cela comme un identifiant plutôt qu'un modèle entraîné.

In [6]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "personalize-poc-userpersonalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_solution_arn = user_personalization_create_solution_response['solutionArn']


In [7]:
print(json.dumps(user_personalization_solution_arn, indent=2))

"arn:aws:personalize:us-east-1:451584754870:solution/personalize-poc-userpersonalization"


#### Créer la version de solution

Une fois que vous avez une solution, vous devez créer une version afin d'achever l'entraînement du modèle. L'entraînement peut prendre un certain temps, plus de 25 minutes, et une moyenne de 90 minutes pour cette recette avec notre jeu de données. Normalement, nous utiliserions une boucle while pour interroger le système jusqu'à ce que la tâche soit terminée. Toutefois, cette tâche bloquerait l'exécution d'autres cellules, et l'objectif ici est de créer de nombreux modèles et de les déployer rapidement. Nous allons donc mettre en place la boucle while pour toutes les solutions plus bas dans ce bloc-notes. Vous y trouverez également des instructions pour visualiser la progression dans la console AWS.

In [7]:
userpersonalization_create_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_solution_arn
)

In [8]:
userpersonalization_solution_version_arn = userpersonalization_create_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-userpersonalization",
  "ResponseMetadata": {
    "RequestId": "11f510f4-f6da-4419-a0e4-567a504c5f9c",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 23:47:58 GMT",
      "x-amzn-requestid": "11f510f4-f6da-4419-a0e4-567a504c5f9c",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### SIMS


SIMS est l'un des algorithmes les plus anciens utilisés dans Amazon pour les systèmes de recommandation. Il est utilisé principalement lorsque vous avez un article et que vous souhaitez recommander des articles qui ont été utilisés de manière similaire par l'ensemble de votre base d'utilisateurs. Cela signifie que le résultat n'est pas personnalisé selon l'utilisateur. Parfois, cette approche implique de recommander principalement des articles populaires. Il existe donc un hyperparamètre qui peut être modifié pour réduire le nombre d'articles populaires dans vos résultats. 

Pour notre cas d'utilisation, en utilisant les données de MovieLens, supposons que nous choisissions un film particulier. Nous pouvons ensuite utiliser SIMS pour recommander d'autres films en fonction du comportement d'interaction de l'ensemble de la base d'utilisateurs. Les résultats ne sont pas personnalisés par utilisateur, mais diffèrent en fonction du film que nous avons choisi en entrée.

Comme la dernière fois, nous commençons par sélectionner la recette.

In [9]:
SIMS_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

#### Créer la solution

Comme pour le HRNN, commencez par créer la solution. Même si vous fournissez le jeu de données ARN dans cette étape, le modèle n'est pas encore entraîné. Considérez cela comme un identifiant plutôt qu'un modèle entraîné.

In [10]:
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-sims",
  "ResponseMetadata": {
    "RequestId": "14c2cab0-6b36-4122-8f71-402acfa0c5e3",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 23:48:12 GMT",
      "x-amzn-requestid": "14c2cab0-6b36-4122-8f71-402acfa0c5e3",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Créer la version de solution

Une fois que vous avez une solution, vous devez créer une version afin d'achever l'entraînement du modèle. L'entraînement peut prendre un certain temps, plus de 25 minutes, et une moyenne de 35 minutes pour cette recette avec notre jeu de données. Normalement, nous utiliserions une boucle while pour interroger le système jusqu'à ce que la tâche soit terminée. Toutefois, cette tâche bloquerait l'exécution d'autres cellules, et l'objectif ici est de créer de nombreux modèles et de les déployer rapidement. Nous allons donc mettre en place la boucle while pour toutes les solutions plus bas dans ce bloc-notes. Vous y trouverez également des instructions pour visualiser la progression dans la console AWS.

In [11]:
sims_create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

In [12]:
sims_solution_version_arn = sims_create_solution_version_response['solutionVersionArn']
print(json.dumps(sims_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-sims/c1a33236",
  "ResponseMetadata": {
    "RequestId": "57e5bcee-b7cf-4eaf-8768-7f81c084232a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 23:48:15 GMT",
      "x-amzn-requestid": "57e5bcee-b7cf-4eaf-8768-7f81c084232a",
      "content-length": "106",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Classement personnalisé

Le classement personnalisé est une application intéressante de HRNN. Au lieu de recommander uniquement ce qui est le plus probable pour l'utilisateur concerné, cet algorithme prend en compte un utilisateur ainsi qu'une liste d'articles. Les articles sont ensuite restitués par ordre de pertinence la plus probable pour l'utilisateur. Le cas d'utilisation ici est le filtrage sur des catégories uniques pour lesquelles vous ne disposez pas de métadonnées sur les articles pour créer un filtre, ou lorsque vous avez une vaste collection que vous souhaitez mieux classer pour un utilisateur particulier.

Dans notre cas d'utilisation, en utilisant les données de MovieLens, nous pourrions imaginer qu'une application de VOD cherche à créer un répertoire de films de bandes dessinées ou de films d'un réalisateur spécifique. Ces listes sont probablement basées sur les métadonnées de titres que nous avons. Nous utiliserions un classement personnalisé pour réorganiser la liste des films pour chaque utilisateur, en fonction de son historique d'étiquetage. 

Comme la dernière fois, nous commençons par sélectionner la recette.

In [13]:
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

#### Créer la solution

Comme pour la solution précédente, commencez par créer la solution. Même si vous fournissez le jeu de données ARN dans cette étape, le modèle n'est pas encore entraîné. Considérez cela comme un identifiant plutôt qu'un modèle entraîné.

In [14]:
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-poc-rerank",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "932af10c-a961-41a1-bd53-079141b054c8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 23:48:26 GMT",
      "x-amzn-requestid": "932af10c-a961-41a1-bd53-079141b054c8",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Créer la version de solution

Une fois que vous avez une solution, vous devez créer une version afin d'achever l'entraînement du modèle. L'entraînement peut prendre un certain temps, plus de 25 minutes, et une moyenne de 35 minutes pour cette recette avec notre jeu de données. Normalement, nous utiliserions une boucle while pour interroger le système jusqu'à ce que la tâche soit terminée. Toutefois, cette tâche bloquerait l'exécution d'autres cellules, et l'objectif ici est de créer de nombreux modèles et de les déployer rapidement. Nous allons donc mettre en place la boucle while pour toutes les solutions plus bas dans ce bloc-notes. Vous y trouverez également des instructions pour visualiser la progression dans la console AWS.

In [15]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

In [16]:
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-rerank/eae37049",
  "ResponseMetadata": {
    "RequestId": "6799b3db-d3ef-4686-8a1f-05b22b42ab28",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 23:48:28 GMT",
      "x-amzn-requestid": "6799b3db-d3ef-4686-8a1f-05b22b42ab28",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Affichez le statut de création de la solution

Voici, comme promis, comment afficher les mises à jour du statut dans la console :

* La console AWS devrait déjà être ouverte dans un autre onglet du navigateur, après l'ouverture de cette instance du bloc-notes. 
* Passez à cet onglet et recherchez en haut le service `Personalize`, puis référez-vous à cette page de service. 
* Cliquez sur `View dataset groups`.
* Cliquez sur le nom de votre groupe de jeux de données, très probablement un nom contenant POC.
* Cliquez sur `Solutions and recipes`.
* Vous voyez maintenant la liste de toutes les solutions que vous avez créées ci-dessus, y compris une colonne avec le statut des versions de la solution. Une fois qu'elle est `Active`, votre solution est prête à être examinée. Elle est également en mesure d'être déployée.

Ou exécutez simplement la cellule ci-dessous pour surveiller le statut de création des versions de la solution.

In [22]:
in_progress_solution_versions = [
    userpersonalization_solution_version_arn,
    sims_solution_version_arn,
    rerank_solution_version_arn
]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

Build succeeded for arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-userpersonalization/236f000b
Build succeeded for arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-rerank/eae37049
At least one solution build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-sims/c1a33236


### Réglage des hyperparamètres

Personalize offre la possibilité d'exécuter un réglage des hyperparamètres lors de la création d'une solution. Cette fonction est désactivée par défaut en raison du calcul supplémentaire nécessaire pour effectuer le réglage des hyperparamètres. Par conséquent, les solutions que nous avons créées ci-dessus utiliseront simplement les valeurs par défaut des hyperparamètres pour chaque recette. Pour en savoir plus sur le réglage des hyperparamètres, reportez-vous à la [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

Si vous avez choisi la bonne recette à utiliser et que vous êtes prêt à lancer le réglage des hyperparamètres, le code suivant montre comment procéder, en utilisant SIMS comme exemple.

```python
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims-hpo",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn,
    performHPO=True
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))
```

Si vous connaissez déjà les valeurs que vous souhaitez utiliser pour un hyperparamètre spécifique, vous pouvez également définir cette valeur lorsque vous créez la solution. Le code ci-dessous montre comment définir la valeur pour le `popularity_discount_factor` pour la recette SIMS.

```python
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims-set-hp",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn,
    solutionConfig = {
        'algorithmHyperParameters': {
            'popularity_discount_factor': '0.7'
        }
    }
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))
```

## Évaluer les versions de la solution <a class="anchor" id="eval"></a>
[Retour au début](#top)

Il ne devrait pas falloir plus d'une heure pour entraîner toutes les solutions de ce bloc-notes. Pendant l'entraînement, nous vous recommandons de prendre le temps de lire en détail les différents algorithmes (recettes) et leur comportement. Le moment est également propice pour envisager des alternatives à la manière dont les données ont été introduites dans le système et le type de résultats que vous attendez.

Lorsque la création des solutions est terminée, l'étape suivante est consacrée à l'obtention des métriques d'évaluation. Personalize calcule ces métriques sur la base d'un sous-ensemble de données d'entraînement. L'image ci-dessous illustre comment Personalize répartit les données. Pour 10 utilisateurs, avec 10 interactions chacun (un cercle représente une interaction), les interactions sont classées de la plus ancienne à la plus récente sur la base de l'horodatage. Personalize utilise toutes les données d'interaction de 90 % des utilisateurs (cercles bleus) pour entraîner la version de la solution, et les 10 % restants pour l'évaluation. Pour chacun des utilisateurs dans les 10 % restants, 90 % de leurs données d'interaction (cercles verts) sont utilisées comme entrée pour le recours au modèle entraîné. Les 10 % restants de leurs données (cercle orange) sont comparés à la solution produite par le modèle et sont utilisés pour calculer les métriques d'évaluation.

![métriques Personalize](static/imgs/personalize_metrics.png)

Nous recommandons de lire [la documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) pour comprendre les métriques, mais nous avons également copié des parties de la documentation ci-dessous pour plus de commodité.

Vous devez comprendre les termes suivants à propos de l'évaluation dans Personalize :

* *Recommandation pertinente* désigne une recommandation qui correspond à une valeur dans les données de test pour l'utilisateur particulier.
* *Rang* désigne la position d'un article recommandé dans la liste des recommandations. La position 1 (haut de la liste) est présumée être la plus pertinente pour l'utilisateur.
* *Requête* désigne l'équivalent interne de l'appel GetRecommendations.

Les métriques fournies par Personalize sont les suivantes :
* **coverage** : La proportion d'articles uniques recommandés à partir de toutes les requêtes sur le nombre total d'articles uniques dans les données de formation (comprend à la fois les jeux de données d'articles et d'interactions).
* **mean_reciprocal_rank_at_25** : La [moyenne des rangs réciproques](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) de la première recommandation pertinente parmi les 25 premières recommandations pour toutes les requêtes. Cette métrique est appropriée si vous êtes intéressé par la recommandation unique la mieux classée.
* **normalized_discounted_cumulative_gain_at_K** : Le gain actualisé suppose que les recommandations situées plus bas dans une liste de recommandations sont moins pertinentes que les recommandations situées plus haut. Par conséquent, chaque recommandation est réduite (pondération plus faible) par un facteur dépendant de sa position. Pour produire le [gain cumulé actualisé](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) à K, toutes les recommandations actualisées pertinentes dans les K premières recommandations sont additionnées. Le gain cumulé actualisé normalisé (NDCG) est le DCG divisé par le DCG idéal de sorte que le NDCG soit compris entre 0 et 1. (Le DCG idéal est celui où les K premières recommandations sont triées par pertinence). Amazon Personalize utilise un facteur de pondération de 1/log(1 + position), où le haut de la liste est la position 1. Cette métrique récompense les articles pertinents qui apparaissent près du haut de la liste, car le haut d'une liste suscite généralement plus d'attention.
* **precision_at_K** : Le nombre de recommandations pertinentes parmi les K premières recommandations divisé par K. Cette métrique récompense la recommandation précise des articles pertinents.

Examinons les métriques d'évaluation de chacune des solutions produites dans ce bloc-notes. *Veuillez noter que vos résultats peuvent varier par rapport aux résultats décrits dans ce bloc-notes, en raison de la qualité du jeu de données Movielens.* 

### Métriques de personnalisation de l'utilisateur

Tout d'abord, récupérez les métriques d'évaluation pour la version de la solution de personnalisation de l'utilisateur.

In [23]:
user_personalization_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = userpersonalization_solution_version_arn
)

print(json.dumps(user_personalization_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-userpersonalization/236f000b",
  "metrics": {
    "coverage": 0.0791,
    "mean_reciprocal_rank_at_25": 0.269,
    "normalized_discounted_cumulative_gain_at_10": 0.2687,
    "normalized_discounted_cumulative_gain_at_25": 0.3321,
    "normalized_discounted_cumulative_gain_at_5": 0.2279,
    "precision_at_10": 0.0571,
    "precision_at_25": 0.0379,
    "precision_at_5": 0.075
  },
  "ResponseMetadata": {
    "RequestId": "fa6f0e49-b87d-48ce-ab27-80e61a1ab395",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:11 GMT",
      "x-amzn-requestid": "fa6f0e49-b87d-48ce-ab27-80e61a1ab395",
      "content-length": "417",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Le gain cumulatif normalisé actualisé ci-dessus nous indique que pour 5 articles, nous avons moins de (38 % pour le jeu complet, 22 % pour le jeu limité) de chance dans une recommandation faisant partie de l'historique d'interaction d'un utilisateur (dans la phase de maintien de l'entraînement et de la validation). Environ 13 % des articles recommandés sont uniques, et nous avons une précision de seulement (14 % pour le jeu complet, 7,5 % pour le jeu limité) dans le top 5 des articles recommandés. 

Ce n'est clairement pas un excellent modèle, mais gardez à l'esprit que nous avons dû utiliser des données d'évaluation pour nos interactions, car Movielens est un jeu de données explicite basé sur les évaluations. Les horodatages remontent également au moment où le film a été évalué, et non regardé, si bien que l'ordre n'est pas le même que celui dans lequel un téléspectateur regarderait des films.

### Métriques SIMS

Maintenant, récupérez les métriques d'évaluation pour la version de la solution SIMS.

In [24]:
sims_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(sims_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-sims/c1a33236",
  "metrics": {
    "coverage": 0.1861,
    "mean_reciprocal_rank_at_25": 0.1669,
    "normalized_discounted_cumulative_gain_at_10": 0.2234,
    "normalized_discounted_cumulative_gain_at_25": 0.2895,
    "normalized_discounted_cumulative_gain_at_5": 0.1889,
    "precision_at_10": 0.0588,
    "precision_at_25": 0.0418,
    "precision_at_5": 0.0824
  },
  "ResponseMetadata": {
    "RequestId": "662598d1-86f0-49a1-98a9-1c3924882f4b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:11 GMT",
      "x-amzn-requestid": "662598d1-86f0-49a1-98a9-1c3924882f4b",
      "content-length": "404",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Dans cet exemple, nous voyons une précision légèrement élevée à 5 articles, un peu plus de (4,5 % pour le jeu complet, 6,4 % pour le jeu limité) cette fois. En fait, la marge d'erreur est probablement respectée. Cependant, puisqu'aucun moyen n'a été utilisé pour masquer la popularité, il se peut que les résultats les plus populaires soient ceux avec lesquels un grand nombre d'utilisateurs ont interagi d'une manière ou d'une autre. 

### Métriques de classement personnalisé

Maintenant, récupérez les métriques d'évaluation pour la version de la solution de classement personnalisé.

In [25]:
rerank_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = rerank_solution_version_arn
)

print(json.dumps(rerank_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-rerank/eae37049",
  "metrics": {
    "coverage": 0.0038,
    "mean_reciprocal_rank_at_25": 0.0537,
    "normalized_discounted_cumulative_gain_at_10": 0.0707,
    "normalized_discounted_cumulative_gain_at_25": 0.0956,
    "normalized_discounted_cumulative_gain_at_5": 0.0578,
    "precision_at_10": 0.0132,
    "precision_at_25": 0.0113,
    "precision_at_5": 0.0189
  },
  "ResponseMetadata": {
    "RequestId": "01f86e05-9408-4456-97d3-2548d2cc998e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:11 GMT",
      "x-amzn-requestid": "01f86e05-9408-4456-97d3-2548d2cc998e",
      "content-length": "406",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Juste un petit commentaire sur celle-ci. Ici, nous voyons à nouveau une précision de près de (2,7 % pour le jeu complet, 2,2 % pour le jeu limité), puisque cette solution est basée sur la personnalisation de l'utilisateur, cela est à prévoir. Cependant, les articles de l'échantillon ne sont pas les mêmes pour la validation, d'où le faible score.

## Utilisation des métriques d'évaluation <a class="anchor" id="use"></a>
[Retour au début](#top)

Il est important d'utiliser soigneusement les métriques d'évaluation. Il y a un certain nombre de facteurs à garder à l'esprit.

* S'il existe un système de recommandation, cela aura influencé l'historique des interactions de l'utilisateur que vous utilisez pour entraîner vos nouvelles solutions. Cela signifie que les métriques d'évaluation sont sélectionnées pour favoriser la solution existante. Si vous œuvrez pour pousser les métriques d'évaluation à correspondre ou à dépasser la solution existante, vous pourriez simplement pousser la personnalisation de l'utilisateur à se comporter comme la solution existante et ne pas finir avec quelque chose de mieux.
* La recette HRNN Coldstart est difficile à évaluer à l'aide des métriques produites par Amazon Personalize. Le but de la recette est de recommander des articles qui sont nouveaux pour votre entreprise. Par conséquent, ces articles n'apparaîtront pas dans les données de transaction utilisateur existantes qui sont utilisées pour calculer les métriques d'évaluation. Par conséquent, lorsqu'elle est comparée aux métriques d'évaluation uniquement, HRNN Coldstart ne donnera jamais l'impression de produire de meilleurs résultats que les autres recettes. Remarque : La recette Personnalisation de l'utilisateur comprend également une fonctionnalité de démarrage à froid améliorée

Compte tenu de ces facteurs, les métriques d'évaluation produites par Personalize sont généralement utiles dans deux cas :
1. Comparer les performances des versions de la solution entraînées sur la même recette, mais avec des valeurs différentes pour les hyperparamètres et les fonctions (données d'impression, etc.).
1. Comparaison des performances des versions de la solution entraînées sur différentes recettes (à l'exception de HRNN Coldstart).

La meilleure façon d'évaluer correctement un système de recommandation est toujours de procéder à des tests A/B tout en mesurant les résultats commerciaux réels. Étant donné que les recommandations générées par un système influencent généralement le comportement de l'utilisateur sur lequel elles sont basées, il est préférable d'effectuer de petites expériences et d'appliquer des tests A/B pendant de plus longues périodes. Au fil du temps, le biais du modèle existant s'estompera.

## Stockage des variables utiles <a class="anchor" id="vars"></a>
[Retour au début](#top)

Avant de quitter ce bloc-notes, exécutez les cellules suivantes pour enregistrer les ARN de la version à utiliser dans le bloc-notes suivant.

In [26]:
%store userpersonalization_solution_version_arn
%store sims_solution_version_arn
%store rerank_solution_version_arn
%store user_personalization_solution_arn
%store sims_solution_arn
%store rerank_solution_arn

Stored 'userpersonalization_solution_version_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'rerank_solution_version_arn' (str)
Stored 'user_personalization_solution_arn' (str)
Stored 'sims_solution_arn' (str)
Stored 'rerank_solution_arn' (str)
